# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [198]:
# imports
from __future__ import print_function

import os
import numpy as np
import pandas as pd
import panel as pn
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv
import ipywidgets
from ipywidgets import interactive
from ipywidgets import FloatSlider
from IPython.display import display
from ipywidgets import interact

%matplotlib inline

In [199]:
# Set up Panel Plotly extension
pn.extension('plotly')

In [200]:
# Import hvplot.pandas after pn.extension
# This avoids plotly initialization failure
import hvplot.pandas

In [201]:
# # Read the Mapbox API key
# load_dotenv()
# mapbox_token = os.getenv("MAPBOX_API_KEY")

# Import Data

In [202]:
# Import the CSVs to Pandas DataFrames
file_path = Path("sfo_neighborhoods_census_data.csv")
# sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data = pd.read_csv(file_path, parse_dates=True, infer_datetime_format=True)
sfo_data.head()

,year,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,2010,Alamo Square,291.182945,372560,1239
1,2010,Anza Vista,267.932583,372560,1239
2,2010,Bayview,170.098665,372560,1239
3,2010,Buena Vista Park,347.394919,372560,1239
4,2010,Central Richmond,319.027623,372560,1239


In [203]:
# Parallel Coordinates
sfo_data_hoodpar = sfo_data[["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"]].groupby(["neighborhood"]).mean().reset_index()
sfo_data_hoodpar.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,Alamo Square,366.020712,378401.0,2817.285714
1,Anza Vista,373.382198,379050.0,3031.833333
2,Bayview,204.588623,376454.0,2318.400000
3,Bayview Heights,590.792839,382295.0,3739.000000
4,Bernal Heights,576.746488,379374.5,3080.333333


In [204]:
# Calculate the mean values for each year
sfo_data_housing = (
    sfo_data[["year", "neighborhood", "housing_units", "sale_price_sqr_foot", "gross_rent"]]
    .groupby(["year", "neighborhood"])
    .mean()
    .sort_values("housing_units")
    .reset_index()
)
sfo_data_housing.head(10)

,year,neighborhood,housing_units,sale_price_sqr_foot,gross_rent
0,2010,Alamo Square,372560,291.182945,1239
1,2010,Mission Bay,372560,489.805378,1239
2,2010,Mission Dolores,372560,257.987999,1239
3,2010,Mission Terrace,372560,262.876026,1239
4,2010,Nob Hill,372560,307.677477,1239
5,2010,Noe Valley,372560,381.138445,1239
6,2010,Oceanview,372560,282.119671,1239
7,2010,Outer Parkside,372560,432.195292,1239
8,2010,Outer Richmond,372560,338.126200,1239
9,2010,Outer Sunset,372560,368.311552,1239


In [205]:
# Calculate the mean values for each year
sfo_data_sales = (
    sfo_data[["year", "housing_units", "sale_price_sqr_foot", "gross_rent"]]
    .groupby("year")
    .mean()
    .sort_values("housing_units")
    .reset_index()
)
sfo_data_sales.head(10)

,year,housing_units,sale_price_sqr_foot,gross_rent
0,2010,372560,369.344353,1239
1,2011,374507,341.903429,1530
2,2012,376454,399.389968,2324
3,2013,378401,483.600304,2971
4,2014,380348,556.277273,3528
5,2015,382295,632.540352,3739
6,2016,384242,697.643709,4390


In [206]:
# Calculate the mean values for each neighborhood
sfo_data_mean = sfo_data[["neighborhood", "sale_price_sqr_foot"]].groupby(["neighborhood"]).mean().reset_index()
sfo_data_mean = sfo_data_mean[["neighborhood", "sale_price_sqr_foot"]].set_index("neighborhood")
sfo_data_mean.dropna().drop_duplicates()
sfo_data_mean.head()

,sale_price_sqr_foot
neighborhood,
Alamo Square,366.020712
Anza Vista,373.382198
Bayview,204.588623
Bayview Heights,590.792839
Bernal Heights,576.746488


In [207]:
sfo_tophoods = sfo_data_mean.loc[sfo_data_mean['sale_price_sqr_foot'] >= 650]
sfo_tophoods.head(20)

,sale_price_sqr_foot
neighborhood,
Cow Hollow,665.964042
Merced Heights,788.844818
Miraloma Park,779.810842
Pacific Heights,689.555817
Potrero Hill,662.013613
Presidio Heights,675.350212
South Beach,650.124479
Telegraph Hill,676.506578
Union Square District,903.993258


In [208]:
# sfo_data.index.names = ['year']
# sfo_data_unit = sfo_data.reset_index()
# sfo_data_unit.dropna().drop_duplicates()
# sfo_data_unit.head()

In [209]:
file_path = Path("neighborhoods_coordinates.csv")
sfo_data_cord = pd.read_csv(file_path)
sfo_data_cord.rename(columns={"Neighborhood": "neighborhood"}, inplace=True)
sfo_data_cord = sfo_data_cord[["neighborhood", "Lat", "Lon"]].set_index("neighborhood")
sfo_data_cord.dropna().drop_duplicates()
sfo_data_cord.head()

,Lat,Lon
neighborhood,,
Alamo Square,37.791012,-122.402100
Anza Vista,37.779598,-122.443451
Bayview,37.734670,-122.401060
Bayview Heights,37.728740,-122.410980
Bernal Heights,37.728630,-122.443050


In [210]:
# Join the average values with the neighborhood locations
hood_scat = (
    pd.concat([sfo_data_cord, sfo_data_mean], axis=1).dropna()
)
hood_scat.index.names = ['neighborhood']
hood_scat = hood_scat.reset_index()
hood_scat.head()

C:\Users\lchri\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:3: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




,neighborhood,Lat,Lon,sale_price_sqr_foot
0,Alamo Square,37.791012,-122.402100,366.020712
1,Anza Vista,37.779598,-122.443451,373.382198
2,Bayview,37.734670,-122.401060,204.588623
3,Bayview Heights,37.728740,-122.410980,590.792839
4,Buena Vista Park,37.768160,-122.439330,452.680591


In [211]:
# # Join the housing units with the neighborhood locations
# hood_sqftunits = (
#     pd.concat([sfo_data_cord, sfo_data_unit], axis=1).dropna()
# )
# hood_sqftunits.index.names = ['neighborhood']
# hood_sqftunits = hood_sqftunits.reset_index()
# hood_sqftunits.head()

In [212]:
# sfo_data.index.names = ['year']
# sfo_data_rent = sfo_data[["gross_rent"]].mean(axis=1)
# sfo_data_rent = sfo_data.reset_index()
# sfo_data_rent.dropna().drop_duplicates()
# sfo_data_rent.head()

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [213]:
# Define Panel Visualization Functions
def average_price_by_neighborhood():
    hood_scatplot = px.scatter_mapbox(
        hood_scat,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="neighborhood",
        color_continuous_scale=px.colors.cyclical.IceFire,
        title="Neighborhood Prices Per/Sqft",
    )
    return hood_scatplot

def neighborhood_map():
    hoodmap_plot = px.scatter_mapbox(
        hood_scat,
        lat="Lat",
        lon="Lon",
        #size="sale_price_sqr_foot",
        color="neighborhood",
        color_continuous_scale=px.colors.cyclical.IceFire,
        title="Neighborhood Map",
    )
    return hoodmap_plot

def average_gross_rent():
    rent_areaplot = px.area(
    sfo_data_housing,
    x="year",
    y="gross_rent",
    color="neighborhood",
    line_group="neighborhood",
    title="Average Gross Rent in San Francisco Per Year",
    )
    return rent_areaplot

def parallel_coordinates():
    hood_parplot = px.parallel_coordinates(
    sfo_data_hoodpar, 
    color="sale_price_sqr_foot",
    title="Parallel Coordinates Plot",
    )
    return hood_parplot

def parallel_categories():
    hood_catplot = px.parallel_categories(
    sfo_data_hoodpar,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={
        "neighborhood": "Neighborhood",
        "sale_price_sqr_foot": "PriceSqft",
        "housing_units": "Housing Units",
        "gross_rent": "Gross Rent"},
    title="Parallel Categories Plot",
)

def housing_units_per_year():
    Hunit_barplot = sfo_data_housing.hvplot.bar(
        x="year",
        y="housing_units",
        ylim=(370000, 390000),
        title="Housing Units Per Year",
        width=1000,
    ).opts(yformatter="%.0f")
    return Hunit_barplot

    
def average_sales_price_year():
    Hood_barplot = sfo_data_sales.hvplot.bar(
        x="year",
        y="sale_price_sqr_foot",
        ylabel="Avg. Sales/yr",
        ylim=(0, 1000),
        title="Average Sales Per Year",
        width=1000,
    ).opts(yformatter="%.0f")
    return Hood_barplot


def top_most_expensive_neighborhoods():
    tophood_barplot = sfo_tophoods.hvplot.bar(
        x="neighborhood",
        y="sale_price_sqr_foot",
        xlabel="neighborhood",
        ylabel="Price/Sqft",
        ylim=(0, 1000),
        title="Top 10 Most Expensive Neighborhoods",
        width=1000,
        rot=90,
        ).opts(yformatter="%.0f")
    return tophood_barplot


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [214]:
# Create panels to structure the layout of the dashboard
geo_sqft_column = pn.Column(
    "## Geo Price Plots", average_price_by_neighborhood(), neighborhood_map()
)

bar_column = pn.Column(
    "## Housing Unit Plots",
    housing_units_per_year(),
    average_sales_price_year(),
    top_most_expensive_neighborhoods(),
)

area_column = pn.Column(
    "## Area Price Plots", average_gross_rent()
)

par_column = pn.Column(
    "## Parallels", parallel_coordinates()
)

# Create tabs
hood_pop_dashboard = pn.Tabs(
    ("Geospatial", geo_sqft_column) ,("Bars", bar_column) ,("Area", area_column), ("Parallels", parallel_coordinates)
)

In [215]:
# Create a scatter mapbox to analyze neighborhood info
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX_API_KEY")

# Set the Mapbox API
px.set_mapbox_access_token(map_box_api)

## Serve the Panel Dashboard

In [216]:
# panel.servable()
# Execute Panel dashboard using servable function
hood_pop_dashboard.servable()

Traceback (most recent call last):
 File "C:\Users\lchri\anaconda3\envs\dev\lib\site-packages\pyviz_comms\__init__.py", line 316, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\lchri\anaconda3\envs\dev\lib\site-packages\panel\viewable.py", line 242, in _on_msg
 patch.apply_to_document(doc, comm.id)
 File "C:\Users\lchri\anaconda3\envs\dev\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 100, in apply_to_document
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\lchri\anaconda3\envs\dev\lib\site-packages\bokeh\document\document.py", line 1150, in _with_self_as_curdoc
 return f()
 File "C:\Users\lchri\anaconda3\envs\dev\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 100, in <lambda>
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\lchri\anaconda3\envs\dev\lib\site-packages\bokeh\document\document.py", line 398, in apply_json_patch
 self._trigger_on_message(event_json["msg_type"], event_json["msg_data"])
 File "C:\Users\lchri\anaconda3\envs\dev\lib\site-packages\bokeh\document\document.py", line 687, in _trigger_on_message
 cb(msg_data)
 File "C:\Users\lchri\anaconda3\envs\dev\lib\site-packages\bokeh\document\document.py", line 348, in apply_json_event
 event = Event.decode_json(json)
 File "C:\Users\lchri\anaconda3\envs\dev\lib\site-packages\bokeh\events.py", line 167, in decode_json
 raise ValueError("Could not find appropriate Event class for event_name: %r" % event_name)
ValueError: Could not find appropriate Event class for event_name: 'document_ready'

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Bars)
        [3] HoloViews(Bars)
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [3] Column
        [0] Column()
        [1] Row
            [0] Plotly(Figure, name='interactive06613')